In [5]:
import os
import sys

import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F  
import matplotlib 
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import torch.utils.data
from torch.utils.data import DataLoader

EPSILON = 0.1       # Max perturbation (for L∞ PGD)
ALPHA = 0.01         # Step size per iteration
ATTACK_ITERATIONS = 40
TARGET_LABEL = 2     # Example target label for the targeted attack

# System/Model parameters
sys.path.append("/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/models")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/weights/best_model_retrained.pth"
#IQ_FILE_PATH = "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/1m_2m_replacedPluto4/Pluto_10_windows_runs2_3/Pluto_10_2m_run3.iq"
IQ_FILE_PATH = "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/1m_2m_replacedPluto4/Pluto_10_windows_runs2_3/Pluto_10_2m_run3.iq"

from attempt2 import resnet50_1d  # Directly import from attempt2.py
num_classes = 8  # Change this if your model was trained with a different number of classes

# Initialize the model architecture
model = resnet50_1d(num_classes=num_classes).to(DEVICE)

# Load trained weights
print(f"Loading trained model weights from: {MODEL_PATH}")
state_dict = torch.load(MODEL_PATH, map_location=DEVICE)

# Load the state dictionary into the model
model.load_state_dict(state_dict)

# Set the model to evaluation mode
model.eval()
print("model is loaded")
class IQDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]

        sample = torch.from_numpy(sample).float()
        # Normalize data
        magnitude = torch.sqrt(torch.sum(sample**2, dim=1, keepdim=True))
        sample = sample / magnitude

        label_tensors = torch.tensor(label, dtype=torch.long)

        return sample, label_tensors


Loading trained model weights from: /home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/weights/best_model_retrained.pth
model is loaded


/tmp/ipykernel_1091547/1990433863.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(MODEL_PATH, map_location=DEVICE)


In [6]:
BATCH_SIZE = 16
WINDOW_SIZE = 10000
HOP_SIZE = 500
START_INDEX = 4800
END_INDEX = 6000

def path_loss(signal, n, d1, d2):
    scaling = (d1 / d2) ** (n / 2)
    return signal * scaling

def apply_rician_fading(signal, K=10.0):
    """
    Apply Rician fading to the signal, which is of shape [2, N] (I and Q).
    Fading is applied using a K-factor (default 10).
    """
    i_window = signal[:, 0, :]
    q_window = signal[:, 1, :]

    device = signal.device
    dtype = signal.dtype
    # Calculate scaling factors
    K = torch.tensor(K, dtype=dtype, device=device)
    scale_LOS = torch.sqrt(K / (K + 1))
    scale_NLOS = torch.sqrt(1 / (K + 1))

    # Generate NLOS (Rayleigh) component
    real_nlos = torch.randn(i_window.shape, device=device) / torch.sqrt(torch.tensor(2.0, dtype=dtype, device=device))
    imag_nlos = torch.randn(i_window.shape, device=device) / torch.sqrt(torch.tensor(2.0, dtype=dtype, device=device))

    # LOS component (typically assumed as 1 + 0j for all samples)
    real_los = torch.ones(i_window.shape, device=device)
    imag_los = torch.zeros(i_window.shape, device=device)

    # Total fading coefficients
    real_fade = scale_LOS * real_los + scale_NLOS * real_nlos
    imag_fade = scale_LOS * imag_los + scale_NLOS * imag_nlos

    # Apply Rician fading
    faded_real = i_window * real_fade - q_window * imag_fade
    faded_imag = i_window * imag_fade + q_window * real_fade

    # Reconstruct the faded signal back into a tensor
    faded_signal = torch.stack((faded_real, faded_imag), dim=1)

    return faded_signal

def apply_awgn(signal, noise_std=0.000001):
    device = signal.device
    dtype = signal.dtype
    # Split noise equally between I and Q (to maintain total variance)
    per_dim_std = noise_std / torch.sqrt(torch.tensor(2.0, dtype=dtype, device=device))
    # Generate i.i.d. Gaussian noise for I and Q
    noise = torch.randn_like(signal, device=device) * per_dim_std
    # Add noise to the signal
    noisy_signal = signal + noise

    return noisy_signal

def transform_channel_effects(x, chosen_distance=2.0, path_loss_exponent=2.0, reference_distance=1.0, noise_std=0.000001, k=10.0):   
    #print("Signal original: ", x)
    signal_path_loss = path_loss(x, path_loss_exponent, reference_distance, chosen_distance)
    #print("Signal Path Loss: ", signal_path_loss)
    signal_rician = apply_rician_fading(signal_path_loss, k)
    #print("Signal Rician: ", signal_rician)
    signal_awgn = apply_awgn(signal_rician, noise_std)
    #print("Signal AWGN: ", signal_awgn)
    return signal_awgn


In [ ]:
def query_limited_nes_attack( # most of the params are unchanged from the previous attacks
    model,
    x,
    perturb,
    y,
    target_label,
    eps=0.1,
    alpha=0.01,
    num_iter=40,
    num_queries=20,     # Number of queries for NES black box limiting
    sigma=0.01,         # Small perturbation scale
    num_samples=10,     # EOT samples (for realism, can be smaller)
    min_distance=1.0,
    max_distance=4.0,
    path_loss_exponent=2.0,
    reference_distance=1.0,
    min_noise_std=0.000001,
    max_noise_std=0.0001,
    min_k=10,
    max_k=20
):
    x_adv = perturb.clone().detach().to(DEVICE) 
    target = torch.full_like(y, target_label).to(DEVICE)

    best_x_adv = x_adv.clone().detach()
    best_target_confidence = -float('inf')

    for i in range(num_iter):
        nes_grad_estimate = torch.zeros_like(x_adv)

        # NES Gradient Estimation via queries
        for _ in range(num_queries): # only prompt in the range of inputted num_queries size
            u = torch.randn_like(x_adv) # small perterbation drawn 
            
            # Evaluate model at (x_adv + sigma*u) using EOT for realism
            logits_plus = 0
            for _ in range(num_samples):
                chosen_distance = torch.empty(1).uniform_(min_distance, max_distance).item()
                chosen_k = torch.empty(1).uniform_(min_k, max_k).item()
                chosen_noise_std = torch.empty(1).uniform_(min_noise_std, max_noise_std).item()
                
                x_t_plus = transform_channel_effects(
                    x_adv + sigma * u,
                    chosen_distance,
                    path_loss_exponent,
                    reference_distance,
                    chosen_noise_std,
                    chosen_k
                ) + x

                logits_plus += model(x_t_plus)
            
            logits_plus /= num_samples #logits_plus is avg model output (logits) across transformations
            loss_plus = F.cross_entropy(logits_plus, target)

            nes_grad_estimate += loss_plus.item() * u

        nes_grad_estimate /= (sigma * num_queries)

        # Update adversarial example using NES gradient approximation
        with torch.no_grad():
            x_adv -= alpha * nes_grad_estimate.sign()
            x_adv = torch.max(torch.min(x_adv, x + eps), x - eps)

        # Evaluate and track best adversarial example
        with torch.no_grad():
            logits_eval = 0
            for _ in range(num_samples):
                chosen_distance = torch.empty(1).uniform_(min_distance, max_distance).item()
                chosen_k = torch.empty(1).uniform_(min_k, max_k).item()
                chosen_noise_std = torch.empty(1).uniform_(min_noise_std, max_noise_std).item()

                x_t_eval = transform_channel_effects(
                    x_adv,
                    chosen_distance,
                    path_loss_exponent,
                    reference_distance,
                    chosen_noise_std,
                    chosen_k
                ) + x

                logits_eval += model(x_t_eval)

            logits_eval /= num_samples
            avg_confidence = F.softmax(logits_eval, dim=1)[0, target_label].item()

            if avg_confidence > best_target_confidence:
                best_target_confidence = avg_confidence
                best_x_adv = x_adv.clone().detach()

            pred = logits_eval.argmax(dim=1)
            success = (pred == target_label).float().mean().item()
            print(f"Iteration {i+1}/{num_iter}: Attack success = {success*100:.2f}%")

    return best_x_adv